In [2]:
import pandas as pd
import numpy as np

In [16]:
aparts = pd.read_csv('./data/aparts.csv')
aparts = aparts[aparts['주소(시도)k-apt주소split'] == '서울'][['k-아파트코드','k-아파트명','주소(시군구)','주소(읍면동)','좌표X','좌표Y']]
aparts = aparts.drop_duplicates(subset='k-아파트코드') # 중복 아파트 제거
aparts = aparts.rename(columns={'좌표X': 'lat', '좌표Y': 'lon'})
aparts

,k-아파트코드,k-아파트명,주소(시군구),주소(읍면동),lat,lon
0,A15679103,우리유앤미,동작구,흑석동,126.959639,37.500668
1,A13876112,송파파인타운13단지,송파구,장지동,127.129179,37.476897
2,A13873701,오금현대백조(임대),송파구,오금동,127.128775,37.508906
3,A15275101,개봉건영,구로구,개봉동,126.840675,37.501162
4,A13991016,월계동원베네스트,노원구,월계동,127.058220,37.631732
...,...,...,...,...,...,...
2794,A10022430,신길AK푸르지오주상복합,영등포구,신길동,126.910099,37.506605
2795,A10022476,논현 e편한세상,강남구,논현동,127.028762,37.506865
2796,A10022525,강동헤리티지자이,강동구,길동,127.148697,37.539149
2797,A10022419,길음역 롯데캐슬 트윈골드,성북구,길음동,127.022976,37.603698


In [41]:
def ROI_facilities(row):
    # 관심영역 설정 3km 이내의 데이터를 뽑아옴
    delta = 0.027  # 약 3km에 해당하는 값 (정확하지 않음)

    min_lat = row['lat'] - delta
    max_lat = row['lat'] + delta
    min_lon = row['lon'] - delta
    max_lon = row['lon'] + delta
    return pd.Series({'min_lat': min_lat, 'max_lat': max_lat, 'min_lon': min_lon, 'max_lon': max_lon})


## 범위내의 시설인지
def in_range(row,aparts_df):
    in_lat_range = (row['lat'] >= aparts_df['min_lat']) & (row['lat'] <= aparts_df['max_lat']) 
    in_lon_range = (row['lon'] >= aparts_df['min_lon']) & (row['lon'] <= aparts_df['max_lon']) 
    return (in_lat_range & in_lon_range).any()



In [31]:
aparts[['min_lat', 'max_lat', 'min_lon', 'max_lon']] = aparts.apply(ROI_facilities, axis=1)


In [32]:
aparts

,k-아파트코드,k-아파트명,주소(시군구),주소(읍면동),lat,lon,min_lat,max_lat,min_lon,max_lon
0,A15679103,우리유앤미,동작구,흑석동,126.959639,37.500668,126.932639,126.986639,37.473668,37.527668
1,A13876112,송파파인타운13단지,송파구,장지동,127.129179,37.476897,127.102179,127.156179,37.449897,37.503897
2,A13873701,오금현대백조(임대),송파구,오금동,127.128775,37.508906,127.101775,127.155775,37.481906,37.535906
3,A15275101,개봉건영,구로구,개봉동,126.840675,37.501162,126.813675,126.867675,37.474162,37.528162
4,A13991016,월계동원베네스트,노원구,월계동,127.058220,37.631732,127.031220,127.085220,37.604732,37.658732
...,...,...,...,...,...,...,...,...,...,...
2794,A10022430,신길AK푸르지오주상복합,영등포구,신길동,126.910099,37.506605,126.883099,126.937099,37.479605,37.533606
2795,A10022476,논현 e편한세상,강남구,논현동,127.028762,37.506865,127.001762,127.055763,37.479865,37.533865
2796,A10022525,강동헤리티지자이,강동구,길동,127.148697,37.539149,127.121697,127.175697,37.512149,37.566149
2797,A10022419,길음역 롯데캐슬 트윈골드,성북구,길음동,127.022976,37.603698,126.995976,127.049976,37.576698,37.630698


In [43]:
cctvs = pd.read_csv('./export/safety/서울시 안심이 CCTV 연계 현황.csv')
cctvs = cctvs.rename(columns={'위도': 'lat', '경도': 'lon'})
cctvs

,자치구,안심 주소,lat,lon,CCTV 수량,수정 일시
0,중랑구,면목3.8동 산16-2 (서일대뒷산공원-화장실)_C-MM38-0001,37.5879,127.0987,1,2022-12-01
1,중랑구,면목3.8동1064-9 (재개발조합사무실)_C-MM38-0022,37.5819,127.0944,1,2022-12-01
2,중랑구,면목3.8동 산16-2 (서일대뒷산공원-체육시설)_C-MM38-0025,37.5876,127.0989,1,2022-12-01
3,중랑구,면목4동 1342-14 (다사랑어린이집앞 공원인도)_C-MM04-0046,37.5717,127.0863,1,2022-12-01
4,중랑구,면목4동 1382 (고정)_C-MM04-0038-A,37.5738,127.0870,1,2022-12-01
...,...,...,...,...,...,...
83729,강남구,대치2-256-02,37.5059,127.0589,1,2022-12-01
83730,강남구,수서-208-01,37.4902,127.1014,1,2022-12-01
83731,강남구,수서-208-02,37.4902,127.1014,1,2022-12-01
83732,강남구,대치2-256-03,37.5059,127.0589,1,2022-12-01


In [45]:
mask = cctvs.apply(lambda row: in_range(row, aparts), axis=1)
filtered_cctvs = cctvs[mask]

In [46]:
filtered_cctvs

,자치구,안심 주소,lat,lon,CCTV 수량,수정 일시
